## Prerequisites
* Download dataset from https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip and unzip.
* Install [PCL](https://pointclouds.org/downloads/), [NumPy](https://numpy.org/install/), [Open3D](http://www.open3d.org/), and [pyquaternion](http://kieranwynn.github.io/pyquaternion/).

## 1. Prepare random uniform downsampled indices of points
Change `root` in `gen_downsample_index.py` according to where the dataset is located. Run it for both splits (train and test).

Note that this step is not a prerequisite for the following data preparation steps.

In [ ]:
%cd my_dataloader
!python gen_downsample_index.py
%cd ..

## 2. Optional: convert txt file into PCD file for LRF computation
txt file could also parsed and used for the LRF computation.

In [ ]:
import open3d as o3d
import numpy as np
import glob

In [ ]:
all_files = glob.glob('modelnet40_normal_resampled/*/*.txt')

for file in all_files:
    f = open(file)
    pc = []
    
    for line in f:
        l = line.split(',')
        pc.append([float(l[0]), float(l[1]), float(l[2])])

    pc = np.array(pc)
    # Create PointCloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc)
    
    o3d.io.write_point_cloud(file[:-3] + "pcd", pcd)

## 3. Compute LRF
PCL implementation of FLARE can be used for the computation of LRFs. See [flare](https://github.com/tolgabirdal/qecnetworks/tree/master/flare) folder in order to use it. All point cloud file names should be saved in `all_pcd_files.txt` before using it.

## 4. Convert LRF orientation to quaternion and find the indices of invalid points

In [ ]:
from pyquaternion import Quaternion

all_lrf_files = [f[:-4] + "_lrf.txt" for f in all_files]
for f in all_lrf_files:
    file = open(f).readlines()
    quat = np.zeros((len(file), 4))
    wrong_ids = []
    for i, line in enumerate(file[1:]):  # assuming that there is a header row in the file
        lrf = line.split(" ")
        
        vecs = np.zeros((3, 3))
        vecs[0] = [float(lrf[0]), float(lrf[1]), float(lrf[2])]  # x
        vecs[1] = [float(lrf[3]), float(lrf[4]), float(lrf[5])]  # y
        vecs[2] = [float(lrf[6]), float(lrf[7]), float(lrf[8][:-1])] # z - don't take last char since it is \n
        
        rotation = vecs.transpose()
        try:
            potential = Quaternion._from_matrix(rotation, rtol=1e-03, atol=1e-03).q
            if all(p < 1e+20 for p in potential):
                quat[i] = potential
            else:
                quat[i] = np.array([0,0,0,0])
                wrong_ids.append(i)
        except:
            quat[i] = np.array([0,0,0,0])
            wrong_ids.append(i)
    np.savetxt(f[:-8] + '.qua', quat, encoding='ascii')
    np.savetxt(f[:-8] + '.idx', np.array(wrong_ids), encoding='ascii')